In [ ]:
import os
import json
import glob
import gzip
import shutil
from tqdm import tqdm
import pickle

import nest_asyncio
nest_asyncio.apply()

import pandas as pd
from deepl import DeepLCLI
import googletrans
import os

## Unzip JSON Files

In [ ]:
# unzip_data_path = glob.glob("../data/c4/realnewslike/*.gz")
# save_dir = "./data/c4"
# for gz_file in unzip_data_path:
#     with gzip.open(gz_file, 'rb') as f_in:
#         file_name = (".").join(gz_file.split("/")[-1].split(".")[:-1])
#         with open(os.path.join(save_dir,file_name), 'wb') as f_out:
#             shutil.copyfileobj(f_in, f_out)

In [ ]:
# os.path.join(save_dir,file_name)

## Merging Files

In [ ]:
# train_files = glob.glob("data/c4_realnewslike/*train*.json")
# valid_files = glob.glob("data/c4_realnewslike/*valid*.json")
# len(train_files), len(valid_files)

In [ ]:
# val_df = pd.read_json(valid_files[0], lines=True)

# train_df = pd.DataFrame()
# for file in tqdm(train_files, desc="merge JSON ..."):
#     tmp = pd.read_json(file, lines=True)
#     train_df = pd.concat([train_df, tmp], axis=0)

In [ ]:
# print(train_df.shape, val_df.shape)

## Save to CSV

In [ ]:
# with open("data/train_total_realnewslike.pkl", "wb") as f:
#     pickle.dump(train_df, f)
#     f.close()

# with open("data/valid_total_realnewslike.pkl", "wb") as f:
#     pickle.dump(val_df, f)
#     f.close()

In [ ]:
# train_df.head()

## Translate with DeepL

In [ ]:
# with open("data/train_total_realnewslike.pkl", "rb") as f:
#     train_df = pickle.load(f)
#     f.close()

# with open("data/valid_total_realnewslike.pkl", "rb") as f:
#     valid_df = pickle.load(f)
#     f.close()

# train_df.head()

## DeepL CLI Configuration

In [ ]:
deepl = DeepLCLI("en", "ko")
# deepl.timeout = 15000
# deepl.max_length = 3000

## Whole Dataset

In [ ]:
def deepl_translate(df):
    deepl = DeepLCLI("en", "ko")
    kor = []
    for idx in tqdm(range(len(df)), desc="Translate ... "):
        if 3000<len(df.iloc[idx,0])<=6000:
            tmp = df.iloc[idx,0].split("\n")

            front = (" ").join(tmp[:len(tmp)//2])
            back = (" ").join(tmp[len(tmp)//2:])

            front_tr = deepl.translate(front)
            back_tr = deepl.translate(back)

            kor.append((front_tr+back_tr).replace("\n", " "))

        elif len(df.iloc[idx,0])>6000:
            continue
        else:
            tr = deepl.translate(df.iloc[idx,0])

            kor.append(tr.replace("\n", " "))
    return kor

In [ ]:
def google_translate(df):
    translator = googletrans.Translator()
    kor = []
    
    for idx in tqdm(range(len(df)), desc="Translate ... "):
        kor.append((translator.translate(df.iloc[idx,0], src="en", dest="ko").text).replace("\n", " "))

In [ ]:
google_translate(valid_df)

In [ ]:
train_pkl = glob.glob("data/merged/train*.pkl")
valid_pkl = "data/merged/validation.00000.pkl"

for i in train_pkl:
    with open(i, "rb") as f:
        train_df = pickle.load(f)
        
    f = open('data/train.txt','a', encoding='utf-8')
    for idx in range(len(train_df)):
        f.write(train_df.iloc[idx, 0])
        f.close()
    
with open(valid_pkl, "rb") as f:
    valid_df = pickle.load(f)

f = open('data/valid.txt','a', encoding='utf-8')
for idx in range(len(valid_df)):
    f.write(valid_df.iloc[idx])
f.close()